<a href="https://colab.research.google.com/github/yf591/llm-toolkit/blob/main/SFT_LLM_FineTuning_GUI_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-3.1-Swallow-8B-Instruct-v0.1の SFTTrainer による QLoRA ファインチューニング

###Overview：
このNotebookは、例としてLlama-3.1-Swallow-8B-Instruct-v0.1モデルをQLoRA（Quantized Low-Rank Adaptation）を用いてファインチューニングする手順を示します。
ファインチューニングには、Hugging Faceの`trl`ライブラリの`SFTTrainer`を使用します。

### 目次
1. **環境セットアップ**
    -  GPUの確認
    -  Googleドライブのマウント
    -  作業ディレクトリの変更
    -  基本パラメータの設定
2. **ライブラリのインストール**
    -  必要なライブラリのインストール
    -  ライブラリのインポート
    -  Hugging Faceへのログイン
3. **モデルの準備**
    -  量子化設定
    -  モデルとトークナイザーのロード
4. **モデルの動作確認**
    -  初期状態での推論テスト
5. **データセットの準備**
    -  データセットのロード
    -  データセットの確認
    -  データセットのフォーマット
6. **QLoRA設定**
    -  線形層の名前の取得
    -  LoRAの設定
7. **学習設定**
    -  学習引数の設定
8. **ファインチューニング**
    -  SFTTrainerによるファインチューニング
9. **学習結果の確認**
    -  学習したパラメータの比率の確認
    -  GPUメモリのリセット
10. **ファインチューニングモデルのロード**
    -  量子化設定
    -  モデルとトークナイザーのロード
    -  ファインチューニングモデルのロード
11. **推論テスト**
    -  文章生成関数の定義
    -  ファインチューニング後のモデルによる推論テスト
12. **Colabランタイムの強制終了（オプション）**

## 1.**環境セットアップ**
-  GPUの確認
-  Googleドライブのマウント
-  作業ディレクトリの変更
-  基本パラメータの設定

In [ ]:
# GPUの状態を確認
!nvidia-smi

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive") # Googleドライブをマウントし、ノートブックからファイルにアクセスできるようにします。

In [ ]:
# 自分のGoogleドライブの作業用フォルダのパスに書き換える
%cd /content/drive/MyDrive/Colab_Notebooks/llm_toolkit_google_colab/01_Instruction_tuning_QLoRA
%ls # 現在のディレクトリにあるファイルを表示

In [ ]:
# 基本パラメータ
model_id = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1" # 使用するモデルのID
peft_name = "Llama3.1-SW-8B-it-v0.1_A100_1rep_qlora" # ファインチューニング後のモデルを保存する際の名前
output_dir = "output_neftune" # 学習済みモデルの出力ディレクトリ

## 2.**ライブラリのインストール**
-  必要なライブラリのインストール
-  ライブラリのインポート
-  Hugging Faceへのログイン

In [ ]:
%%time
# ライブラリのインストール
!pip install peft # PEFT（Parameter-Efficient Fine-Tuning）ライブラリ
!pip install transformers==4.43.3
!pip install datasets==2.14.5
!pip install accelerate bitsandbytes evaluate
!pip install trl==0.12.0 # TRL（Transformer Reinforcement Learning）ライブラリ,（バージョン0.12.0を指定）
!pip install flash-attn # T4では使えないのでコメントアウト

In [ ]:
# ライブラリーのインストール

import torch # PyTorchライブラリ
from torch import cuda, bfloat16 # PyTorchのCUDA, bfloat16関連の機能
from transformers import ( # Transformersライブラリから必要なクラスをインポート
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)


from datasets import load_dataset # Datasetsライブラリからload_dataset関数をインポート
from peft import LoraConfig, PeftModel # PEFTライブラリから必要なクラスをインポート
from trl import SFTTrainer # TRLライブラリからSFTTrainerをインポート

In [ ]:
from huggingface_hub import login # Hugging Face Hubのlogin関数をインポート
from google.colab import userdata # Google Colabのuserdataモジュールをインポート

# HuggingFaceアカウントにログイン
login(userdata.get('HF_TOKEN')) # Colabのシークレットキーを使用（Hugging Faceのトークンを設定しておく必要があります。）

## 3.**モデルの準備**
-  量子化設定
-  モデルとトークナイザーのロード

In [ ]:
# 量子化設定 (量子化に関する設定)
bnb_config = BitsAndBytesConfig( # 量子化設定用のオブジェクトを作成開始
    load_in_4bit=True,           # モデルを4ビットで読み込む設定 (メモリ削減)
    bnb_4bit_use_double_quant=True, # 4ビット量子化で二重量子化を使用 (精度向上)
    bnb_4bit_quant_type="nf4",     # 4ビット量子化のタイプをNF4形式に指定
    bnb_4bit_compute_dtype=torch.bfloat16 # 量子化中の計算精度をbfloat16に指定
)

# モデルの設定 (モデル読み込みの設定)
model = AutoModelForCausalLM.from_pretrained( # 事前学習済みモデルをロード開始
    model_id,                      # 使用するモデルのID (Hugging Face Hub上の名前など)
    trust_remote_code=True,        # リモート(Hub上)のカスタムコード実行を許可
    quantization_config=bnb_config,# 上記で定義した量子化設定を適用
    device_map='auto',             # モデルをGPU/CPUに自動で割り当て
    torch_dtype=torch.bfloat16,    # モデルの計算時のデータ型をbfloat16に指定
    attn_implementation="flash_attention_2" # 高速化技術FlashAttention2
)

# tokenizerの設定 (トークナイザー読み込みの設定)
tokenizer = AutoTokenizer.from_pretrained( # 事前学習済みトークナイザーをロード開始
    model_id,                      # 使用するトークナイザーのID (モデルと通常同じ)
    padding_side="right",          # パディング(穴埋め)をシーケンスの右側に行う設定
    add_eos_token=True             # 文末に終了を示すEOSトークンを自動で追加する設定
)

if tokenizer.pad_token_id is None: # もしパディング用トークンIDが未設定なら
  tokenizer.pad_token_id = tokenizer.eos_token_id # 終了トークンIDをパディング用IDとして使う


## 4.**モデルの動作確認**
- 初期状態での推論テスト

In [ ]:
# テスト用のメッセージを作成 (ここからモデルへの入力メッセージを作成)
messages = [                     # チャット形式のメッセージリストを作成開始
    {"role": "system", "content": "あなたは常に日本語で応答する優秀なアシスタントです。"}, # システムメッセージ(AIの役割指示)
    {"role": "user", "content": "広島県の美味しい食べ物や有名な建造物は何ですか？"},         # ユーザーからの質問メッセージ
] # メッセージリスト作成完了

# 入力メッセージをトークン化し、モデルのデバイスに転送 (ここから入力データをモデル用に変換)
input_ids = tokenizer.apply_chat_template( # チャットテンプレートを適用してトークンIDに変換開始
    messages,                    # 変換するメッセージリスト
    add_generation_prompt=True,  # AIの応答を促すプロンプトを追加する設定
    return_tensors="pt"          # 結果をPyTorchテンソル形式で返す設定
).to(model.device)               # 変換結果をモデルと同じデバイス(GPU等)に移動

# 文章生成を終了するトークンIDを設定 (ここで生成停止の条件を設定)
terminators = [                  # 生成終了のトリガーとなるトークンIDのリストを作成開始
    tokenizer.eos_token_id,      # 標準の終了(EOS)トークンID
    tokenizer.convert_tokens_to_ids("<|eot_id|>") # 特定の終了用トークン文字列をIDに変換して追加
] # 生成終了トークンIDリスト作成完了

# モデルを使用して文章を生成 (ここから実際に文章を生成)
outputs = model.generate(        # モデルの`generate`メソッドで文章生成を開始
    input_ids,                   # 生成の元となる入力トークンID
    max_new_tokens=256,          # 生成する新しいトークン数の上限を256に設定
    eos_token_id=terminators,    # 生成停止のトリガーとなるトークンID(リスト)を指定
    do_sample=True,              # 次のトークンを確率的にサンプリングする方式を使う
    temperature=0.8,             # サンプリングのランダム性を調整 (低いほど決定的)
    top_p=0.8,                   # Top-pサンプリングを使用 (累積確率0.8までの候補から選ぶ)
    pad_token_id=tokenizer.eos_token_id, # 生成中に使うパディングトークンIDを指定 (EOSと同じ)
    attention_mask=torch.ones(input_ids.shape, dtype=torch.long).cuda(), # 入力部分全体に注意を向けるマスクを作成しGPUへ
) # 文章生成完了、結果を`outputs`に格納

# 生成されたトークンからレスポンスを抽出 (ここから生成結果の後処理)
response = outputs[0][input_ids.shape[-1]:] # 生成結果(outputsの最初の要素)から入力部分を除いた応答部分を抽出

# 生成されたレスポンスを整形して表示 (ここから結果を読みやすく表示)
import textwrap                  # テキストの折り返し用ライブラリをインポート
s = tokenizer.decode(response, skip_special_tokens=True) # 応答部分のトークンIDを文字列にデコード (特殊トークンは削除)
s_wrap_list = textwrap.wrap(s, 50) # デコードした文字列を50文字ごとに折り返してリスト化
print('\n'.join(s_wrap_list))    # 折り返した文字列リストを改行で繋げて表示



## 5.**データセットの準備**
-  データセットのロード
-  データセットの確認
-  データセットのフォーマット

In [ ]:
# ローカル（MyDrive上）にあるデータセットをロード
dataset = load_dataset("./dataset", split="train") # データセットは、このNotebookが実行されるディレクトリにdatasetという名前のフォルダがある想定

In [ ]:
# データセットの中身確認
dataset[200]

In [ ]:
# データセットの各要素を、チャット形式のメッセージに変換する関数
def formatting_func(example):
        messages = [
            {'role': "system",'content': "あなたは日本語で回答するアシスタントです"},
            {'role': "user",'content': example["instruction"]},
            {'role': "assistant",'content': example["output"]}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False)


# データセットの各要素を更新する関数
# フォーマットされたテキストを"text"キーに追加し、不要なキーを削除
def update_dataset(example):
    example["text"] = formatting_func(example)
    for field in ["index", "category", "instruction", "input", "output"]:
        example.pop(field, None)
    return example


# データセットを更新
dataset = dataset.map(update_dataset)

# 更新されたデータセットの11番目の要素の"text"を表示
print(dataset[10]["text"])

## 6.**QLoRA設定**
-  線形層の名前の取得
-  LoRAの設定

In [ ]:
# モデルの情報を表示
model

In [ ]:
# モデルから（4ビット量子化された）線形層の名前を取得する関数
import bitsandbytes as bnb

# モデルのすべての線形層の名前を取得する関数
def find_all_linear_names(model):
    target_class = bnb.nn.Linear4bit
    linear_layer_names = set()
    for name_list, module in model.named_modules():
        if isinstance(module, target_class):
            names = name_list.split('.')
            layer_name = names[-1] if len(names) > 1 else names[0]
            linear_layer_names.add(layer_name)
    if 'lm_head' in linear_layer_names:
        linear_layer_names.remove('lm_head')
    return list(linear_layer_names)

target_modules = find_all_linear_names(model) # 線形層の名前を取得
print(target_modules) # 線形層の名前を表示

In [ ]:
peft_config = LoraConfig(
    r=8, # LoRAのランク
    lora_alpha=16, # LoRAのアルファ値
    lora_dropout=0.05, # LoRAのドロップアウト率
    target_modules = target_modules, # LoRAを適用するモジュール
    bias="none", # バイアス項を使用しない
    task_type="CAUSAL_LM", # タスクの種類
    modules_to_save=["embed_tokens"], # 学習後に保存するモジュール
)


## 7.**学習設定**
-  学習引数の設定

In [ ]:
# 学習中の評価、保存、ロギングを行う間隔を設定
eval_steps = 20
save_steps = 20
logging_steps = 20

# 学習引数の設定
training_arguments = TrainingArguments(
    bf16=True, # bfloat16を使用
    per_device_train_batch_size=4, # デバイスごとのバッチサイズ
    gradient_accumulation_steps=16, # 勾配累積ステップ数
    num_train_epochs=1, # 学習エポック数
    optim="adamw_torch_fused", # 最適化アルゴリズム
    learning_rate=2e-4, # 学習率
    lr_scheduler_type="cosine", # 学習率スケジューラ
    weight_decay=0.01, # 重み減衰
    warmup_steps=100, # ウォームアップステップ数
    group_by_length=True, # 長さでグループ化
    report_to="none", # wandbへのレポートを無効化
    logging_steps=logging_steps, # ログの記録間隔
    eval_steps=eval_steps, # 評価間隔
    save_steps=save_steps, # モデルの保存間隔
    output_dir=output_dir, # 学習済みモデルの出力ディレクトリ
    save_total_limit=3, # 保存するモデルの最大数
    push_to_hub=False, # Hugging Face Hubへのアップロードを無効化
    # report_to="wandb"
    auto_find_batch_size=True # GPUメモリのオーバーフロー防止（バッチサイズを自動で調整）
)



## 8.**ファインチューニング**
-  SFTTrainerによるファインチューニング

In [ ]:
# SFTTrainerの初期化
trainer = SFTTrainer(
    model=model, # モデル
    tokenizer=tokenizer, # トークナイザー
    train_dataset=dataset, # 学習データセット
    dataset_text_field="text", # データセットのテキストフィールド
    peft_config=peft_config, # PEFTの設定
    args=training_arguments, # 学習引数
    max_seq_length=1024, # 最大シーケンス長
    packing=True, # パッキングを使用
    neftune_noise_alpha=5, # NEFTune設定, NEFTuneノイズアルファ値
)

# wandb.init(project="llama3_sftqlora")

In [ ]:
%%time

# 学習中に警告を抑制するためにキャッシュを使用しない設定にし、学習後にキャッシュを有効にする。
model.config.use_cache = False
trainer.train()
model.config.use_cache = True

# 学習したQLoRAモデルを保存
trainer.model.save_pretrained(peft_name)

## 9.**学習結果の確認**
-  学習したパラメータの比率の確認
-  GPUメモリのリセット

In [ ]:
# 学習したパラメータの比率確認
trainer.model.print_trainable_parameters()

In [ ]:
import torch
torch.cuda.empty_cache() # GPUメモリをリセット

## 10.**ファインチューニングモデルのロード**
-  量子化設定
-  モデルとトークナイザーのロード
-  ファインチューニングモデルのロード

In [ ]:
%%time
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# モデルの設定・ロード
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)


# tokenizerの設定・ロード
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side="right",
    add_eos_token=True
)
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# ファインチューニングモデルの作成
from peft import PeftModel
ft_model = PeftModel.from_pretrained(model, peft_name)

## 11.**推論テスト**(1～3のいずれかで実施)
-  文章生成関数の定義
-  ファインチューニング後のモデルによる推論テスト

### 11.1 コードベースでテスト

In [ ]:
# 文章生成関数を定義
def generate(prompt):
  # 入力プロンプトをチャット形式のメッセージに変換
  messages = [
      {"role": "system", "content": "あなたは日本語で回答するアシスタントです。"},
      {"role": "user", "content": prompt},
  ]

  # 入力メッセージをトークン化し、モデルのデバイスに転送
  input_ids = tokenizer.apply_chat_template( # This line had an extra indent
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(ft_model.device)

  # 文章生成を終了するトークンIDを設定
  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  # モデルを使用して文章を生成
  outputs = ft_model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      pad_token_id=tokenizer.eos_token_id, # 追加
      attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(ft_model.device),
  )

  # 生成されたトークンからレスポンスを抽出
  response = outputs[0][input_ids.shape[-1]:]

  # print(tokenizer.decode(response, skip_special_tokens=True))

  # 生成されたレスポンスを整形して表示
  import textwrap
  s = tokenizer.decode(response, skip_special_tokens=True)
  s_wrap_list = textwrap.wrap(s, 50) # 50字で改行したリストに変換
  print('\n'.join(s_wrap_list))

In [ ]:
%%time

# ファインチューニングされたモデルを使用して文章を生成
generate("こんにちは。最近の調子はどうですか？")

In [ ]:
%%time
generate("CPUとGPUの違いは何ですか？ 詳しく教えてください。")

In [ ]:
%%time
generate("広島では何が有名ですか？")

In [ ]:
%%time
generate("広島にあるプロスポーツチームを教えて？")

###11.2 Gradio を利用

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# --- 文章生成関数 (GUI対応版) ---
def generate(prompt):
    messages = [
        {"role": "system", "content": "あなたは日本語で回答するアシスタントです。"},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(ft_model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = ft_model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(ft_model.device),
    )

    response = outputs[0][input_ids.shape[-1]:]
    s = tokenizer.decode(response, skip_special_tokens=True)
    return s

In [ ]:
# --- Gradio GUI ---
iface = gr.Interface(
    fn=generate,
    inputs=gr.Textbox(lines=5, placeholder="ここにプロンプトを入力してください"),
    outputs=gr.Textbox(),
    title="Llama-3.1-Swallow-8B-Instruct-v0.1 Fine-tuned Model",
    description="ファインチューニングされたモデルで文章を生成します。"
)

# GUIの起動
iface.launch(share=True)

## 12.**Colabランタイムの強制終了（オプション）**

In [ ]:
# # Colabラインタイムの強制終了（オプション）
# from google.colab import runtime
# runtime.unassign()

## Reference
- [QLORA:Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)
- [【Llama3】SFTTrainerで簡単ファインチューニング(QLoRA)](https://highreso.jp/edgehub/machinelearning/llama3sftqlora.html)
- [huggingface/TRLのSFTTrainerクラスを使えばLLMのInstruction Tuningのコードがスッキリ書けてとても便利です](https://qiita.com/m__k/items/23ced0db6846e97d41cd)
- [TRL - Transformer Reinforcement Learning](https://huggingface.co/docs/trl/index)
- [Google Colabによる Llama3.2 / Qwen2.5 の ファインチューニング・ハンズオン](https://www.youtube.com/watch?v=fp4GC6OUZGc)